# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [3]:
%pip install numpy
import numpy as np
print("NumPy version:", np.__version__)
print("Import successful!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 1.4 MB/s  0:00:04m0:00:0100:01m
NumPy version: 2.3.3
Import successful!


In [6]:
%pip install dotenv
%pip install openai

from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

Note: you may need to restart the kernel to use updated packages.
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp313-cp313-macosx_10_12_x86_64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 947.6/947.6 kB 11.5 MB/s  0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached pydantic_core-2.33.2-cp313-cp313-macosx_10_12_x86_64.whl (2.0 MB)
Using cache

In [7]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [8]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [9]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [10]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [ ]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [11]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #1.2!


##### ✅ Answer:
1. You can pass in a value to the dimensions paramater to reduce the default of 1536 dimensions.
2. This change only happens for the `text-embedding-3-small` and `text-embedding-3-large` models - you can reduce the number of dimensions and you will not lose any concept-representing properties, unlike other models. Ideally, you should modify this value before creating the embedding. They are able to acheive this by encoding information dynamically and having individual embeddings adapt to new constraints as it is created.

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [12]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer:
Creating requests synchronously mean the work will be done one at a time. Only till the first request finishes can you move on to the second. An asychronous approach means that the requests are done parallelly. If the first request is taking too long, you can work on the second and third one at the same time. 
Some benefits of using an `async` approach include:
1. If you have a large number of texts (in the hundreds or thousands), an `async` approach lets you make many requests concurrently. A `synchronous` approach means you have to wait for one request to finish before moving on to the next. This would take too long for a large data set. A synchronous approach might work if you have a very small data set - i.e. 5-10 queries total.
2. Async scales better. Since you are not limited by the order in which things are done, you can work with other heavy operations like API calls. This also helps web servers scale because you need to serve many users at the same time.
3. An async approach means faster processing of large batches and more efficient resource usage. 


So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [13]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer:
Prompt Engineering - writing effective instructions for the model so that the output has clear success criteria and meets your requirements more consistently.
There are some techniques that work with any model, such as using message roles. However, the prompts may need to be adjusted based on the model. Even different snapshots of models within the same family can produce different results with the same prompts. For that reason, it is recommended to pin your applications to model snapshots for consistent behavior. And to build a way to evaluate the behavior and performance of the prompts so you update it as necessary.
You can pass in different insructions which behave like certain roles (developer). You can also pass in prompts as certain roles. The `developer` role has the highest priority and are used as instructions for the model. The `user` role is prioritized after `developer`, and these are the instructions given by the user - the query asked. There is also the `assistant` role which are messages generated by the model.
You can also pass in reusable prompts which can be updated without changing any of the code. This allows you to iterate until you consistently get successful responses.


### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [14]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [15]:
print(response)

Hello! The best way to write a loop in Python depends on what you want to accomplish, but generally, using a `for` loop is very clear and Pythonic when iterating over sequences like lists, strings, or ranges.

Here's a simple example using a `for` loop:

```python
for i in range(5):
    print(i)
```

This will print numbers from 0 to 4.

If you need to loop until a certain condition is met and don't know the exact number of iterations in advance, a `while` loop might be more suitable:

```python
count = 0
while count < 5:
    print(count)
    count += 1
```

Both loops are perfectly fine, but favoring `for` loops when you have a sequence or known range often leads to clearer and more readable code.

If you'd like, I can help you write a loop for a specific task you have in mind! Just let me know.


## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [17]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable and thoughtful assistant that provides deep analysis based on provided context.

Before answering, ask yourself these questions:
- What is the core issue being described?
- What causes or factors contribute to this situation?
- What are the consequences or implications?
- What patterns or principles does this illustrate?
- How might this apply in similar situations?

Instructions:
- Think through your response step-by-step before providing your final answer
- First, analyze what the context tells us about the topic
- Then, identify the key insights and implications
- Finally, synthesize this into a comprehensive response
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Show your reasoning process before giving your final answer

Then provide a comprehensive response that addresses these analytical dimensions while staying strictly within the provided context.

Format your response as:
**Analysis:** [Your step-by-step thinking]
**Answer:** [Your final comprehensive response]"""


RAG_USER_TEMPLATE = """Context Information: {context}
Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please analyze this question by considering:
1. What does the context directly state about this topic?
2. What are the underlying principles or patterns described?
3. What implications or consequences are mentioned or can be inferred?
4. How does this connect to broader concepts mentioned in the context?

Provide your answer based solely on the context above, showing your analytical process."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "detailed",
        "response_length": "comprehensive"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [18]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [19]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

Response: **Analysis:**  
1. The context from Source 1 explicitly defines the "Michael Eisner Memorial Weak Executive Problem" as a tendency seen in CEOs or startup founders who previously excelled in a specific function (e.g., product management, sales, marketing) but, upon becoming CEO, hire weak executives to run that same function. This happens because the CEO struggles to let go and wants to remain "the man" or the key figure in that area, effectively undermining the strength of the executive team. The name derives from Michael Eisner’s example: he had been a brilliant TV network executive before becoming Disney’s CEO, yet after Disney bought ABC, the network fell to fourth place. Eisner reportedly said he could turn it around if he had two extra days a week, but he did not, illustrating a CEO’s limitations in trying to micromanage or directly control areas they once mastered.

2. The underlying principle or pattern is that a CEO who is deeply attached to their original function m

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer:
The overall strategy to improve LLM responses is advanced or analytical prompt engineering techniques. All of these smaller strategies together will improve the responses and consistency of success.
1. Chain of thought processing. Have the LLM think step by step and show it's reasoning before outputing the final response.
2. Right now the LLM isn't allowed to use any external information. If we want a more accurate response, we should allow for different paths of reasoning which includes information that wasn't explicitly given to it by the user. We can also have the system think about the problem in different ways and then converge the ideas at the end for a more comprehensive response.
3. We tell the LLM to `respond with "I don't know"` if it doesn't know the answer to a query. We can have the system ask itself questions to create more in depth reasoning for each question.
4. We can assign more well defined roles to the system so it understands what we expect from the responses better.

### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database
- Use a different embedding model
- Add the capability to ingest a YouTube link

While these are suggestions, you should feel free to make whatever augmentations you desire! If you shared an idea during Session 1, think about features you might need to incorporate for your use case! 

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

Now on to Youtube

In [20]:
%pip install youtube-transcript-api

Note: you may need to restart the kernel to use updated packages.


In [24]:
from youtube_transcript_api import YouTubeTranscriptApi
from urllib.parse import urlparse, parse_qs

def get_video_id(url: str) -> str:
    parsed_url = urlparse(url)
    # Extract query parameters
    query_params = parse_qs(parsed_url.query)
    return query_params.get("v", [None])[0]

def get_video_transcript(video_id: str, lang: str = "en") -> list:
    print("video_id", video_id)
    youtube_api = YouTubeTranscriptApi()
    transcript = youtube_api.fetch(video_id, languages=[lang])
    # transcript is a list of dicts: [{'text': '...', 'start': 0.0, 'duration': 5.0}, ...]
    return [{'text': snippet.text, 'start': snippet.start, 'duration': snippet.duration} for snippet in transcript]

def chunk_transcript(transcript, chunk_size=120):
    chunks, current_chunk, current_start = [], [], 0

    for entry in transcript:
        start_time = entry['start']
        if start_time - current_start < chunk_size:
            current_chunk.append(entry['text'])
        else:
            chunks.append((current_start, " ".join(current_chunk)))
            current_chunk = [entry['text']]
            current_start = start_time
    if current_chunk:
        chunks.append((current_start, " ".join(current_chunk)))
    return chunks

def summarize_chunks(chunks, llm):
    summaries = []
    for start_time, text in chunks:
        response = llm.run([
            {"role": "system", "content": "You are a helpful assistant that summarizes YouTube transcripts."},
            {"role": "user", "content": f"Summarize this segment:\n\n{text}"}
        ])
        summaries.append((start_time, response))
    return summaries

def format_summary(summaries):
    for start_time, summary in summaries:
        minutes = int(start_time // 60)
        print(f"[{minutes:02d}:00] {summary}")




In [26]:
# youtube_url = "8vFGrNjT4P4"
youtube_url =input("Youtube Url: ")
video_id = get_video_id(youtube_url)
print(f"URL, {video_id}")


transcript = get_video_transcript(video_id)

print(f"transcript, {transcript}")

chunked_transcript = chunk_transcript(transcript)

print(f"chunked_transcript, {chunked_transcript}")



URL, 8vFGrNjT4P4
video_id 8vFGrNjT4P4
transcript, [{'text': 'Just about everything you could build\nstarts with knowing how to make a basic', 'start': 0.03, 'duration': 3.99}, {'text': "box. Whether you're making cabinets or\ndrawers or bookcases, desks, beds, they're", 'start': 4.02, 'duration': 6.27}, {'text': "all just variations on the humble box.\xa0\nThis is a great first project if you're", 'start': 10.29, 'duration': 6.03}, {'text': "just getting into woodworking. And\nthere's lots and lots of ways to make", 'start': 16.32, 'duration': 3.24}, {'text': "boxes with all different kinds of\njoinery but in this video I'll show you", 'start': 19.56, 'duration': 4.2}, {'text': "the simple rabbet joint method I use\nalmost every time. It's quick. It's strong.", 'start': 23.76, 'duration': 5.79}, {'text': 'It gives you nice square corners without\na lot of fuss. Hey welcome to Woodworking', 'start': 29.55, 'duration': 4.38}, {'text': "for Mere Mortals where it's my goal to\ntake the mys

In [27]:
summarized_chunks = summarize_chunks(chunked_transcript, chat_openai)

print(f"summarized_chunks, {summarized_chunks}")


formatted_summary = format_summary(summarized_chunks)

print(f"formatted_summary, {formatted_summary}")

summarized_chunks, [(0, 'The video emphasizes that nearly all woodworking projects start with building a basic box, such as cabinets, drawers, or bookcases, making it a great beginner project. The presenter introduces the simple rabbet joint method for constructing boxes, highlighting its speed, strength, and ability to create square corners easily. The demonstration uses standard three-quarter inch lumber available at home centers and mentions that smaller boxes look better with thinner lumber, which will be covered in another video. Free plans for the exact box shown are available in the description. The construction process starts by crosscutting the sides to length, then ripping them to equal widths, cleaning one side before ripping for accuracy. The two lid pieces are glued and lightly clamped to dry, ensuring they fit well together without buckling by checking the edge for straightness after clamping.'), (120.03, 'The speaker is gluing up a panel and is using several clamps along